<a href="https://colab.research.google.com/github/ccwu0918/yolov7-colab/blob/main/Mediapipe_Hands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usage example of MediaPipe Hands Solution API in Python (see also http://solutions.mediapipe.dev/hands).

In [ ]:
!pip install mediapipe

Upload any image that contains hand(s) to the Colab. We took two examples from the web: https://unsplash.com/photos/QyCH5jwrD_A and https://unsplash.com/photos/mt2fyrdXxzk


In [ ]:
!wget -q -O image.jpg https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/woman_hands.jpg

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import math
import numpy as np

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2_imshow(img)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in uploaded.keys()}
# Preview the images.
for name, image in images.items():
  print(name)
  resize_and_show(image)

All MediaPipe Solutions Python API examples are under mp.solutions.

For the MediaPipe Hands solution, we can access this module as `mp_hands = mp.solutions.hands`.

You may change the parameters, such as `static_image_mode`, `max_num_hands`, and `min_detection_confidence`, during the initialization. Run `help(mp_hands.Hands)` to get more informations about the parameters.

In [ ]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
help(mp_hands.Hands)

In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [ ]:
# # 步驟一：導入必要函式庫
# import mediapipe as mp
# from mediapipe.tasks import python
# from mediapipe.tasks.python import vision

# # 步驟二：建立手部特徵點物件
# # 載入手部特徵點偵測模型
# base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
# # 建立手部特徵點偵測基本參數
# options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
# # 建立手部特徵點偵測器
# detector = vision.HandLandmarker.create_from_options(options)

# # 步驟三：載入測試影像（可自行修改成待測試影像名稱）
# image = mp.Image.create_from_file('kira-auf-der-heide-QyCH5jwrD_A-unsplash.jpg')

# # 步驟四：偵測手部特徵點
# detection_result = detector.detect(image)

# # 步驟五：產生結果影像並顯示
# # 將偵測到的結果（特徵點及連結線）繪製到新影像上
# annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# # 顯示時需將色彩格式RGB轉回BGR才能正確顯示。
# cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# For static images:
IMAGE_FILES = ['image.jpg']
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imwrite(
        './annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    resize_and_show(cv2.flip(annotated_image, 1))

In [ ]:
# Run MediaPipe Hands.
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.7) as hands:
  for name, image in images.items():
    # Convert the BGR image to RGB, flip the image around y-axis for correct
    # handedness output and process it with MediaPipe Hands.
    results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

    # Print handedness (left v.s. right hand).
    print(f'Handedness of {name}:')
    print(results.multi_handedness)
    # print(results.multi_handedness[0].classification[0].label)

    if not results.multi_hand_landmarks:
      continue
    # Draw hand landmarks of each hand.
    print(f'Hand landmarks of {name}:')
    image_height, image_width, _ = image.shape
    annotated_image = cv2.flip(image.copy(), 1)
    for hand_landmarks in results.multi_hand_landmarks:
      # Print index finger tip coordinates.
      print(
          f'Index finger tip coordinate: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    resize_and_show(cv2.flip(annotated_image, 1))

In [ ]:
# Run MediaPipe Hands and plot 3d hands world landmarks.
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.7) as hands:
  for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Hands.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # Draw hand world landmarks.
    print(f'Hand world landmarks of {name}:')
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

In [ ]:
!pip install colabcam

In [ ]:
import colabcam
colabcam.take_photo("1.jpg")

## 安裝MQTT套件

```
!pip install paho-mqtt
```
或者
```
!pip3 install paho-mqtt
```

In [ ]:
!pip install paho-mqtt
# 或者
# !pip3 install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt #import the client1
import json
import time
import random
import ssl
import certifi

# 設置日期時間的格式
ISOTIMEFORMAT = '%y-%m-%d %H:%M:%S'

# MQTT
mqtt_client_name = ""
# mqtt_host = "broker.emqx.io"
mqtt_host = "mqttgo.io"
# mqtt_host = "mqtt.xplatform.tranx.io"
# mqtt_ssl_host = "mqttssl.xplatform.tranx.io"

# discovery_prefix = API_Key
discovery_prefix = "ccwu0918"
# component = 'sensor'
node_id = 'video'
# node_id = 'video'
mqtt_topic_template = discovery_prefix + "/"  + node_id

def on_log(client, userdata, level, buf):
  print("log: ", buf)

# 連線設定
# 初始化地端程式
client = mqtt.Client(mqtt_client_name)

# 設定登入帳號密碼
# client.username_pw_set("User_Name", "User_Password")
# client.username_pw_set(User_Name, API_Key)

# client.on_log = on_log

# 設定 SSL/TLS 加密連線
# client.tls_set()
# client.tls_set("/content/certificate.pem.crt", tls_version=ssl.PROTOCOL_TLSv1_2)
# client.tls_set(certifi.where())
# client.tls_insecure_set(True)

# 設定連線資訊(IP, Port, 連線時間)
# client.connect("mqtt.xplatform.tranx.io", 1883, 60)
# client.connect(mqtt_ssl_host, port=8883, keepalive=60)
client.connect(mqtt_host, port=1883, keepalive=60)

# while True:
#   mqtt_topic = str( mqtt_topic_template)
#   payload_dict = {}
#   Temperature = random.randint(20, 30)
#   payload_dict['Temperature'] = Temperature
#   Humidity = random.randint(50, 70)
#   payload_dict['Humidity'] = Humidity
#   payload_json = json.dumps(payload_dict)
#   mqtt_payload = str(payload_json)

#   print("mqtt_topic:" + mqtt_topic)
#   print("mqtt_payload:" + mqtt_payload)

#   client.publish(mqtt_topic, mqtt_payload)

#   # 等待10秒發送一次資料
#   # print("sleep 10 seconds.")
#   time.sleep(10)
#   # print("printed after 10 seconds.")

In [ ]:
#Note:bbox是另外疊加顯示的，速度會有延遲是正常的
import cv2
import time
import math
import mediapipe as mp
import colabcam
import numpy as np

# For webcam input:
# cap = cv2.VideoCapture(0)

############## 各參數設定 ##################
pTime  = 0
minPwm = 0
maxPwm = 255
briArd = 0
briBar = 400
briPer = 0

# start streaming video from webcam
colabcam.video_stream()
# label for video
label_html = '顯示中...(點擊畫面以結束顯示)'
# initialze bounding box to empty
bbox = ''
zeros = np.zeros((480, 640, 1))

title = '' # 存放手勢名稱及置信度字串
gesture_name = '' #存放手勢名稱

with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while True:
        js_reply = colabcam.video_frame(label_html, bbox)
        if not js_reply: break

        # while cap.isOpened():
        # success, image = cap.read()
        # if not success:
        #   print("Ignoring empty camera frame.")
        #   # If loading a video, use 'break' instead of 'continue'.
        #   continue

        # convert JS response to OpenCV Image
        image = colabcam.js_to_image(js_reply["img"])

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        # image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        results = hands.process(image)
        # results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
        # gesture_name = results.multi_handedness[0].classification[0].label

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        frame_height, frame_width = np.shape(image)[0:2]

        # tmpImg = np.zeros([frame_height, frame_width, 4], dtype=np.uint8)
        overlapImg = np.zeros([frame_height, frame_width, 3], dtype=np.uint8)

        # 計算每秒跑幾張
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        lmList = []

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                for id, lm in enumerate(hand_landmarks.landmark):
                    # print(id, lm)
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    # print(id, cx, cy)
                    lmList.append( [id, cx, cy])

                    # if draw:
                    #     cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

                    # print('hand_landmarks:', hand_landmarks)
                    # print(
                    #     f'Index finger tip coordinates: (',
                    #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                    #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                    # )

                # mp_drawing.draw_landmarks(
                #    overlapImg, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                mp_drawing.draw_landmarks(
                    overlapImg,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
            # cv2.imshow('MediaPipe Hands', image)
            # overlapImg = cv2.rectangle(overlapImg,(40,40),(60,60),(255,0,0),2)
            # cv2.putText(overlapImg,'text test',(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            # resize_and_show(cv2.flip(overlapImg, 1))

            if len(lmList) != 0:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2]
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

                cv2.circle(overlapImg, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
                cv2.circle(overlapImg, (x2, y2), 15, (255, 0, 255), cv2.FILLED)

                #計算大拇指和食指的直線中點距離
                cv2.line(overlapImg, (x1, y1), (x2, y2), (255, 0, 255), 3)
                cv2.circle(overlapImg, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

                #計算大拇指和食指的直線距離
                length = math.hypot(x2 - x1, y2 - y1)
                #print(length)

                #將大拇指和食指的直線距離換算成0~255，Arduino PWM控制數值亦為0~255
                brightness = np.interp(length, [50, 300], [minPwm, maxPwm])

                briBar = np.interp(length, [50, 300], [400, 150])
                briArd = np.around(brightness,2)


                if length < 50:
                    cv2.circle(overlapImg, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

            #畫出直方圖
            cv2.rectangle(overlapImg,(50,150),(85,400),(255, 0, 255),3)
            cv2.rectangle(overlapImg,(50,150),(85,400),(255, 0, 255),3)
            cv2.rectangle(overlapImg,(50, int(briBar)),(85,400),(255, 0, 255),cv2.FILLED)
            cv2.putText(overlapImg, f'brightness: {int(briArd)}', (15, 140), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 255), 3)

        # 將BGR影像轉換為BGRA格式
        bgra = cv2.cvtColor(overlapImg, cv2.COLOR_BGR2BGRA)
        cv2.putText(bgra, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)
        bgra[:,:,3] = (bgra.max(axis = 2) > 0 ).astype(int) * 0
            # tmp = np.concatenate((overlapImg, np.zeros((480, 640, 1))))

        ret, data = cv2.imencode('.jpg', bgra)   # compress array of pixels to JPG data
        # data = b64encode(data)                  # encode base64
        # data = data.decode()

        mqtt_payload = data.tobytes()
        # mqtt_payload = bytearray(data)
        mqtt_topic = "ccwu0918/video"

        # print("mqtt_topic:" + mqtt_topic)
        # print("mqtt_payload:" + str(mqtt_payload))

        client.publish(mqtt_topic, mqtt_payload)

        mqtt_topic = "ccwu0918/brightness"
        client.publish(mqtt_topic, briArd)

        # mqtt_topic = "ccwu0918/gesture"
        # client.publish(mqtt_topic, gesture_name)

        # 等待10秒發送一次資料
        # print("sleep 10 seconds.")
        time.sleep(1)
        # print("printed after 10 seconds.")

        bbox = colabcam.cvImg2bbox(bgra)

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    # image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    # image.flags.writeable = False
    # results = hands.process(image)

    # # Draw the hand annotations on the image.
    # image.flags.writeable = True
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # if results.multi_hand_landmarks:
    #   for hand_landmarks in results.multi_hand_landmarks:
    #     mp_drawing.draw_landmarks(
    #         image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # cv2.imshow('MediaPipe Hands', image)
    # if cv2.waitKey(5) & 0xFF == 27:
    #   break